In [1]:
import os
import json

import pandas as pd

#### Load

json, etc. https://www.realpythonproject.com/a-cheat-sheet-for-working-with-json-data-in-python/

In [53]:
# FIXING DUMB DUMP BUG
"""
DATA_DIR = os.path.join(os.getcwd(), 'data')
DATA_FILES = list(map(
                    lambda f: os.path.join(DATA_DIR, f), os.listdir(DATA_DIR)
                ))

for fp in DATA_FILES:
    filename = fp.split('/')[-1]
    NEW_DATA_PATH = os.path.join(os.getcwd(), 'data', f'fix__{filename}')
    with open(fp, 'r') as rf, open(NEW_DATA_PATH, 'w') as wf:
        tmp = rf.readlines()
        data = []
        
        wf.write('[')
        for line in tmp:
            if line[:2] == '}{':
                data.append('},{\n')
            else:
                data.append(line)

        wf.writelines(data)
        wf.write(']')
"""

In [ ]:
# Load one (sample)
DATA_PATH = os.path.join(os.getcwd(), 'data', 'fix__2020-04-20__0.json')

with open(DATA_PATH, 'r') as f:
    tmp = json.load(f)
    df = pd.DataFrame(tmp)

df.head()

In [3]:
# Load all
DATA_DIR = os.path.join(os.getcwd(), 'data')
DATA_FILES = list(map(
                    lambda f: os.path.join(DATA_DIR, f), os.listdir(DATA_DIR)
                ))

df = pd.DataFrame()

for fp in DATA_FILES:
    with open(fp, 'r') as f:
        tmp = json.load(f)
        tmp = pd.DataFrame(tmp)
    
    df = pd.concat([df, tmp])
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99975 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   author_id            99975 non-null  int64  
 1   id                   99975 non-null  int64  
 2   in_reply_to_user_id  29159 non-null  float64
 3   created_at           99975 non-null  object 
 4   lang                 99975 non-null  object 
 5   text                 99975 non-null  object 
 6   possibly_sensitive   99975 non-null  bool   
 7   source               99975 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 6.2+ MB


#### Exploration

In [4]:
df.head()

,author_id,id,in_reply_to_user_id,created_at,lang,text,possibly_sensitive,source
0,711679239,1516700055759118337,208429848.0,2022-04-20 08:47:26.000000,und,@Crypto_Niazi #FEG,False,Twitter for iPhone
1,1376829402122575872,1516700055759114244,241214642.0,2022-04-20 08:47:26.000000,en,@dexamol_ @NoCreative_eth @Paddy_Stash @HChipN...,False,Twitter Web App
2,1463462196499804162,1516700055641935873,NaN,2022-04-20 08:47:26.000000,en,"“When you have a dream, you’ve got to grab it ...",False,Twitter Web App
3,1500281789797236737,1516700055537094656,NaN,2022-04-20 08:47:26.000000,en,The rumours are not true. We will not be prici...,False,Twitter Web App
4,1187027027637014528,1516700055427854340,NaN,2022-04-20 08:47:26.000000,en,I just listed this bored ape NFT for 20 ETH in...,False,Twitter for Android


In [164]:
df.lang.unique()

array(['en'], dtype=object)

In [6]:
len(df.source.unique()), df.source.unique()[:10]

(778,
 array(['Twitter for iPhone', 'Twitter Web App', 'Twitter for Android',
        'TwitterTestingApp7', 'abnormal_crypto_app', 'TweetApp for iPhone',
        'IFTTT', 'TweetApp for Android', 'Tweep+', 'Twitter'], dtype=object))

In [7]:
# remove duplicate
df = df[~df.text.duplicated()]

In [8]:
df.created_at = pd.to_datetime(df.created_at)
df.created_at

0      2022-04-20 08:47:26
1      2022-04-20 08:47:26
2      2022-04-20 08:47:26
3      2022-04-20 08:47:26
4      2022-04-20 08:47:26
               ...        
9982   2022-04-20 07:07:53
9989   2022-04-20 07:07:53
9990   2022-04-20 07:07:53
9994   2022-04-20 07:07:53
9999   2022-04-20 07:07:53
Name: created_at, Length: 51569, dtype: datetime64[ns]

In [157]:
df.created_at.min()

Timestamp('2022-04-20 06:22:10')

#### Cleaning

In [83]:
sum(df.lang != 'en')

# skip translate; kelamaan.... (13k ~ 4 jam)

13536

In [89]:
df = df[df.lang == 'en']
df.shape

(38033, 9)

##### Emoji

In [111]:
import re
import string

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator as GoogleTranslator

In [12]:
# From
# https://stackoverflow.com/a/58356570/8996974
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [52]:
def print_5(df):
    for t in df.text.iloc[:5]:
        print(t)
        
print_5(df)        

@Crypto_Niazi #FEG
@dexamol_ @NoCreative_eth @Paddy_Stash @HChipNFT @kukulabanze This is such a cool concept!
“When you have a dream, you’ve got to grab it and never let go.”
@sapeofficialeth is my dream what’s your’s 
https://t.co/e8pvUvkd3J
#bsape  #sape #safuape #crypto #eth #btc @sapeofficialeth https://t.co/IEc0awVfns https://t.co/vPoi5M4yUb
The rumours are not true. We will not be pricing @SmartPiggyNFT at 0.25 $ETH... if you guess our Mint Price you get a free NFT and $25 in ETH #HotMama 😱
.
.
.
#NFTCommunity #NFTsales #NFTgiveaway #NFTAlpha #NFTGiveaways #FreeNFT #FreeNFTs #fearthepiggy #smartpiggy #trending https://t.co/rCBPCtcooq
I just listed this bored ape NFT for 20 ETH in Opensea https://t.co/UQyWo73uI2


In [53]:
df.text = df.text.apply(remove_emojis)

In [54]:
print_5(df)

@Crypto_Niazi #FEG
@dexamol_ @NoCreative_eth @Paddy_Stash @HChipNFT @kukulabanze This is such a cool concept!
“When you have a dream, you’ve got to grab it and never let go.”
@sapeofficialeth is my dream what’s your’s 
https://t.co/e8pvUvkd3J
#bsape  #sape #safuape #crypto #eth #btc @sapeofficialeth https://t.co/IEc0awVfns https://t.co/vPoi5M4yUb
The rumours are not true. We will not be pricing @SmartPiggyNFT at 0.25 $ETH... if you guess our Mint Price you get a free NFT and $25 in ETH #HotMama 
.
.
.
#NFTCommunity #NFTsales #NFTgiveaway #NFTAlpha #NFTGiveaways #FreeNFT #FreeNFTs #fearthepiggy #smartpiggy #trending https://t.co/rCBPCtcooq
I just listed this bored ape NFT for 20 ETH in Opensea https://t.co/UQyWo73uI2


##### Text

[Contekan](https://github.com/bayhaqy/analisa-sentimen)

In [69]:
# From
# https://github.com/ssut/py-googletrans/issues/301#issuecomment-888261389
from time import sleep
import backoff

class Translator:
    def __init__(self):
        self.client = GoogleTranslator()
        self.sleep_in_between_translations_seconds = 1
        self.source_language = "en"
        self.max_chunk_size = 4000

    def __createChunks(self, corpus):
        chunks = [corpus[i:i + self.max_chunk_size] for i in range(0, len(corpus), self.max_chunk_size)]
        return chunks

    def __sleepBetweenQuery(self):
        print('Sleeping for {}s after translation query..'.format(self.sleep_in_between_translations_seconds))
        sleep(self.sleep_in_between_translations_seconds)

    @backoff.on_exception(backoff.expo, Exception, max_tries=150)
    def Translate(self, content, dest_language_code):
        try:
            print('Attempting to translate to lang={}'.format(dest_language_code))
            if len(content) > self.max_chunk_size:
                print('Warning: Content is longer than allowed size of {}, breaking into chunks'.format(self.max_chunk_size))
                results_list = []
                concatenated_result = ""

                original_chunks = self.__createChunks(content)
                for i in original_chunks:
                    r = self.client.translate(i, dest=dest_language_code, src=self.source_language)
                    self.__sleepBetweenQuery()
                    results_list.append(r.text)

                for i in results_list:
                    concatenated_result += i

                return concatenated_result
            else:
                res = self.client.translate(content, dest=dest_language_code, src=self.source_language)
                self.__sleepBetweenQuery()
                return res.text
        except Exception as e:
            print(e)
            raise e

In [132]:
# Replace karakter berulang
def hapus_katadouble(tweet):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", tweet)

def clean_tweet(tweet):
    # Case folding
    tweet = tweet.lower()
  
    # Cleansing (Remove URL)
#     tweet = re.sub('http\S+|\S+co\S+', ' ', tweet)
    tweet = re.sub('http\S+|\S+co\S+', '', tweet)
    
    # Cleansing (Remove Mention)
    tweet = re.sub("@[A-Za-z0-9\S]+", "", tweet)
  
    # Cleansing (Remove Hastag)
#     tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub(r'#([^\s]+)', r'', tweet)
    
    # Cleansing (Remove Number and Punctuation)
    wrem_list = ('rt',)
    exclude = set (string.punctuation)
    rem_list = []
    token = tweet.split()
    for w in token:
        if w not in wrem_list:
            for x in w:
                if x in exclude or x.isdigit():
                    rem_list.append("")
                else:
                    rem_list.append(x)
            rem_list.append(" ")
    tweet = "".join(rem_list)
  
    tweet = hapus_katadouble(tweet)
  
    # hapus extra space dalam kalimat
    return " ".join(tweet.split())


# masukkan stopord tambahan
STOP_TAMBAH = None
extra_stopwords = os.path.join(os.getcwd(), 'extras', 'stop_tambah.txt')
with open(extra_stopwords) as f:
    tmp = f.readlines()
    STOP_TAMBAH = set(map(lambda x: x.rstrip('\n'), tmp))

# Preprocessing english
def preprocessing_en(tweet):

    # PorterStemmer English dari library NLTK
    stemmer = PorterStemmer()
    #stemmer = WordNetLemmatizer()

    token_words = word_tokenize(tweet)
    sentence = []
    for word in token_words:
        if word not in STOP_TAMBAH and len(word) > 1 and len(word) < 25:   
            sentence.append(stemmer.stem(word))
            #sentence.append(stemmer.lemmatize(word,pos='v'))
    return sentence
  

# Translate ke English menggunakan Google Translate
def gtrans_tweet_en(tweet, lang):
    if not tweet:
        return None
    
    if lang == "en":
        return tweet
    """
    try:
        lang = GoogleTranslator().detect(tweet).lang
    except:
        return None
    """
    
    translator = Translator()
    text = translator.Translate(tweet, dest='en').text
    
    return text


# Klasifikasi Polaritas Tweet menggunakan Vader
def sentiment_Vader(tweet):
    analysis = SentimentIntensityAnalyzer()
    analysis = analysis.polarity_scores(tweet)
    comm = analysis['compound']
    if (comm >= 0.05):
        return "Positive"
    elif ((comm > -0.05) and (comm < 0.05)):
        return "Neutral"
    else:
        return "Negative"

# Cek analisis sentimen dari English tweet menggunakan VADER
def sentiment_analysis_en(tweet):
    tweet = clean_tweet(tweet)
    
    # NO PREPROCESSING HAPPENS HERE
    
    tweet = gtrans_tweet_en(tweet)
    sentiment = sentiment_Vader(tweet)
    return sentiment

In [59]:
# sample
clean_tweet(df.text.iloc[3])

'the rumours are not true we will not be pricing at eth if you guess our mint price you get a free nft and in eth'

In [ ]:
# all in one, not preprocessed
# df['sentiment'] = df.text.apply(sentiment_analysis_en)

In [73]:
df['clean_tweet'] = df.text.apply(clean_tweet)

In [119]:
# discard empty string
df = df[df.clean_tweet != ""]

In [109]:
# SKIP TRANSLATE
# sample = df.iloc[0:10].copy()
# sample['clean_tweet_en'] = sample.apply(lambda x: gtrans_tweet_en(x['clean_tweet'], x['lang']), axis=1)
%timeit df['clean_tweet_en'] = df.apply(lambda x: gtrans_tweet_en(x['clean_tweet'], x['lang']), axis=1)

591 ms ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [130]:
sample = sample = df.iloc[0:10000].copy()

In [140]:
"""5 detik per 10k data"""
# %timeit df['preprocessed'] = df.clean_tweet_en.apply(preprocessing_en)
start = time.time()
sample['preprocessed'] = sample.clean_tweet_en.apply(preprocessing_en)
end = time.time()
print(end - start)

4.592994689941406


In [ ]:
# df['preprocessed'] = df.clean_tweet_en.apply(preprocessing_en)
# df['sentiment_clean_en'] = df.clean_tweet_en.apply(sentiment_Vader)

In [138]:
"""2 menit per 10k data"""
# %timeit df['sentiment_clean_en'] = df.clean_tweet_en.apply(sentiment_Vader)
start = time.time()
sample['sentiment_clean_en'] = sample.clean_tweet_en.apply(sentiment_Vader)
end = time.time()
print(end - start)

121.03955078125


In [143]:
sample.head()

,author_id,id,in_reply_to_user_id,created_at,lang,text,possibly_sensitive,source,clean_tweet,clean_tweet_en,preprocessed,sentiment_clean_en
1,1376829402122575872,1516700055759114244,241214642.0,2022-04-20 08:47:26,en,@dexamol_ @NoCreative_eth @Paddy_Stash @HChipN...,False,Twitter Web App,this is such a cool concept,this is such a cool concept,"[cool, concept]",Positive
2,1463462196499804162,1516700055641935873,NaN,2022-04-20 08:47:26,en,"“When you have a dream, you’ve got to grab it ...",False,Twitter Web App,“when you have a dream you’ve got to grab it a...,“when you have a dream you’ve got to grab it a...,"[dream, grab, never, dream]",Positive
3,1500281789797236737,1516700055537094656,NaN,2022-04-20 08:47:26,en,The rumours are not true. We will not be prici...,False,Twitter Web App,the rumours are not true we will not be pricin...,the rumours are not true we will not be pricin...,"[rumour, not, true, will, not, price, eth, if,...",Positive
4,1187027027637014528,1516700055427854340,NaN,2022-04-20 08:47:26,en,I just listed this bored ape NFT for 20 ETH in...,False,Twitter for Android,i just listed this bored ape nft for eth in op...,i just listed this bored ape nft for eth in op...,"[list, bore, ape, nft, eth, opensea]",Negative
5,1502908260659179521,1516700055423623172,NaN,2022-04-20 08:47:26,en,"Fuck it. Drop your $ETH wallets\n\nRT, Like\nF...",False,Twitter for Android,fuck it drop your eth wallets rt like follow me,fuck it drop your eth wallets rt like follow me,"[fuck, drop, eth, wallet, follow]",Negative


In [152]:
CHECKPOINT_PATH = os.path.join(os.getcwd(), 'extras', 'checkpoint.csv')
sample.to_csv(CHECKPOINT_PATH, index=False)

#### Feature Engineering

#### Modeling

#### Eval

#### Testing

#### Viz